# SQL 고득점 Kit

## SELECT


### 모든 레코드 조회
```
SELECT *
FROM ANIMAL_INS
```



### 역순 정렬하기
```
SELECT name, datetime
FROM animal_ins
ORDER BY animal_id DESC
```



### 아픈 동물 찾기
```
SELECT animal_id, name
FROM animal_ins
where intake_condition='sick'
```



### 동물의 아이디와 이름
```
SELECT animal_id, name
from animal_ins
order by animal_id ASC
```

### 여러 기준으로 정렬하기

```
SELECT animal_id, name, datetime
from animal_ins
order by name asc, datetime desc
```



### 상위 n개 레코드

```
SELECT name
from ANIMAL_INS 
order by datetime asc
limit 1
```



## SUM, MAX, MIN


### 최댓값 구하기
```
SELECT max(datetime)
from animal_ins
```


### 최솟값 구하기
```
SELECT min(datetime)
from animal_ins
```


### 동물 수 구하기
```
SELECT count(animal_id) as count
from animal_ins
```


### 중복 제거하기
```
SELECT count(distinct(name)) as count
from animal_ins
```
NULL 값은 count하지 않음
- count(column)하면 NULL 제외하고 개수 반환
- count(*)하면 NULL도 포함해서 개수 반환

하지만, 빈 문자열은 COUNT() 결과에 포함됩니다. 빈 문자열을 COUNT() 결과에서 빼려면 빈 문자열을 NULL로 바꾸고 COUNT()를 하면 됩니다.

- mysql : COUNT( IF (ColumnName='' NULL, ColumnName) )
- mssql : COUNT( CASE WHEN ColumnName='' THEN NULL ELSE ColumnName END )

TRIM() 함수를 사용하면 스페이스, 탭, 줄바꿈 등의 의미 없는 문자만 있는 것도 COUNT 되지 않게 됩니다.

COUNT( DISTINCT( ColumnName ) )으로 하면 중복되지 않고 NULL은 제외되는 값들만 COUNT 하게 됩니다.

참고 : https://benant.wordpress.com/2010/07/25/sql%EC%97%90%EC%84%9C-count-%EA%B2%B0%EA%B3%BC%EC%97%90-null%EC%9D%B4-%ED%8F%AC%ED%95%A8%EB%90%A0%EA%B9%8C/

## GROUP BY

### 고양이와 개는 몇 마리 있을까


```
SELECT animal_type, count(*)
from animal_ins
group by animal_type
order by animal_type ASC

```
```
SELECT animal_type, count(animal_type) as count
from animal_ins
group by animal_type
order by animal_type ASC
```


### 동명 동물 수 찾기

```
SELECT NAME, count(NAME) as COUNT
from animal_ins
group by NAME 
having count(NAME)>1 
order by NAME ASC
```


```
SELECT name, count(name)
from animal_ins
where name is not null
group by name 
having count(name)>1
order by name ASC
```




### 입양 시각 구하기(1)

```
SELECT hour(datetime) as HOUR, count(datetime) as COUNT
from animal_outs
where hour(datetime) >= 9 and  hour(datetime) < 20
group by HOUR
order by HOUR ASC
```
```
SELECT hour(datetime) as HOUR, count(datetime) as COUNT
from animal_outs
group by HOUR
having HOUR >= 9 and  HOUR < 20
order by HOUR ASC
```

SELECT - 5순위 (필수)

FROM - 1순위 (필수)

WHERE - 2순위

GROUP BY - 3순위

HAVING - 4순위

ORDER BY - 6순위

참고 : https://chanhuiseok.github.io/posts/db-5/


### 입양 시각 구하기(2)


```
# 쿼리에서 로컬 변수 활용
SET @hour := -1; # 변수 선언, @변수명 = 프로시저가 종료되어도 유지, := 대입연산

SELECT (@hour := @hour+1) as hour, (SELECT count(*) from animal_outs where HOUR(datetime) = @hour) as COUNT
from animal_outs
where @hour < 23
```
https://chanhuiseok.github.io/posts/db-6/

## IS NULL

### 이름이 없는 동물의 아이디

```
SELECT animal_id
from animal_ins
where name is null
```


### 이름이 있는 동물의 아이디

```
SELECT animal_id
from animal_ins
where name is not null
```


### NULL 처리하기

```
SELECT animal_type, 
       case
       when name is null then 'No name'
       else name
       end as name,
       sex_upon_intake
from animal_ins
```
```
SELECT animal_type, 
       ifnull(name,"No name") as name,
       sex_upon_intake
from animal_ins
```
```
SELECT animal_type, 
       coalesce(name,"No name") as name,
       sex_upon_intake
from animal_ins
```


## JOIN

## 없어진 기록 찾기

```
SELECT o.animal_id, o.name
from animal_outs as o
left join animal_ins as i on o.animal_id = i.animal_id
where i.animal_id is null
```
https://chanhuiseok.github.io/posts/db-7/


## 있었는데요 없었습니다

```
SELECT i.animal_id, i.name
from animal_outs as o, animal_ins as i
where i.animal_id = o.animal_id and o.datetime < i.datetime
order by i.datetime
```

```
SELECT i.animal_id, i.name
from animal_ins as i
join animal_outs as o on i.animal_id = o.animal_id
where o.datetime < i.datetime
order by i.datetime
```



##오랜 기간 보호한 동물(1)


```
SELECT i.name, i.datetime
from animal_ins as i
left join animal_outs as o on i.animal_id = o.animal_id 
where o.animal_id is null
order by i.datetime
limit 3
```


## 보호소에서 중성화한 동물


```
SELECT i.animal_id, i.animal_type, i.name
from animal_ins as i
join animal_outs as o on o.animal_id = i.animal_id 
where i.sex_upon_intake in ('Intact Male','Intact Female') and o.sex_upon_outcome in ('Spayed Female', 'Neutered Male')

```


## String, Date

## 루시와 엘라 찾기

```
SELECT animal_id, name, sex_upon_intake
from animal_ins
where name in ('Lucy', 'Ella', 'Pickle', 'Rogan', 'Sabrina', 'Mitty')
order by animal_id 
```



## 이름에 el이 들어가는 동물 찾기


```
SELECT animal_id, name
from animal_ins
where name like '%el%' and animal_type = 'dog'
order by name
```

SQL에서 LIKE절 사용법

'-' : 글자 숫자를 정해줌 (컬럼명 LIKE '루_시')

'%' : 글자 숫자를 정해주지 않고 해당 문자를 포함한 결과를 모두 출력한다  ( 컬럼명 LIKE '루%')

## 중성화 여부 파악하기

```
SELECT animal_id, name, 
  case
    when sex_upon_intake like 'Neutered%' then 'O'
    when sex_upon_intake like 'Spayed%' then 'O'
  else 'X' end as '중성화'
from animal_ins
order by animal_id 
```






## 오랜 기간 보호한 동물(2)


```
SELECT outs.animal_id, outs.name
from animal_outs outs, animal_ins ins
where outs.animal_id = ins.animal_id
order by outs.datetime-ins.datetime DESC
limit 2
```


## DATETIME에서 DATE로 형 변환


```
SELECT animal_id, name, DATE_FORMAT(datetime, '%Y-%m-%d') as '날짜'
from animal_ins
order by animal_id
```

